In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [12]:
# Load data
df = pd.read_csv("/Users/mariahloehr/IICD/IICD/Data/cell_cycle_tidied.csv")

df['phase'] = df['phase'].replace({'M': 'G2'})

# Separate features and target
X = df.drop(columns=['phase', 'age', 'PHATE_1', 'PHATE_2'])  # exclude phase and age
y = df['phase']  # target is now phase

# Split data into train and test sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=949, stratify=y)

In [13]:
# Define RBF-kernel SVM
svm = SVC(kernel='rbf', probability=False, random_state=949)

In [18]:
# Hyperparameter tuning
param_grid = {
    'C': [200, 300, 400],
    'gamma': [0.0001, 0.00001]
}

grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Output best settings
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

Best parameters: {'C': 300, 'gamma': 0.0001}
Best cross-validation accuracy: 0.9556497175141242


In [19]:
# Retrain using best params
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, y_train)

# Predict
y_train_pred = best_svm.predict(X_train)
y_test_pred = best_svm.predict(X_test)

In [20]:
# Output results
print("\n=== Training Set ===")
print("Overall Accuracy:", accuracy_score(y_train, y_train_pred))

print("\n=== Test Set ===")
print("Overall Accuracy:", accuracy_score(y_test, y_test_pred))

# Accuracy per class
df_train = pd.DataFrame({'true': y_train, 'pred': y_train_pred})
accuracy_per_phase_train = df_train.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
print("Accuracy per phase (Train):")
print(accuracy_per_phase_train)

df_test = pd.DataFrame({'true': y_test, 'pred': y_test_pred})
accuracy_per_phase_test = df_test.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
print("Accuracy per phase (Test):")
print(accuracy_per_phase_test)

# Confusion Matrix
print("\nTest Confusion Matrix")
print(confusion_matrix(y_test, y_test_pred, labels=["G0", "G1", "G2", "S"]))


=== Training Set ===
Overall Accuracy: 0.9844632768361582

=== Test Set ===
Overall Accuracy: 0.9587570621468926
Accuracy per phase (Train):
true
G0    0.984605
G1    0.981840
G2    0.980042
S     0.989529
dtype: float64
Accuracy per phase (Test):
true
G0    0.973190
G1    0.952681
G2    0.941176
S     0.963810
dtype: float64

Test Confusion Matrix
[[363   7   2   1]
 [  7 604   4  19]
 [  1   2 224  11]
 [  1   7  11 506]]


/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_71145/3019480901.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_per_phase_train = df_train.groupby('true').apply(lambda x: accuracy_score(x['true'], x['pred']))
/var/folders/1s/bvxr71hj0hqgyk_jk6k7wkm80000gn/T/ipykernel_71145/3019480901.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_per_phase_test = df_test.groupby('true').apply(